In [424]:
#import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#from bs4 import BeautifulSoup
import pandas as pd
import re
import time
#mport pickle
import datetime
import os
import pickle
import logging



In [425]:

logging.basicConfig(filename='debug.log',level=logging.INFO)
debug = False
url = "https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/{}"


In [426]:

driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
driver.get("https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/1")


In [30]:
#for p in driver.find_elements_by_xpath("//*[@id='results']/div[2]/table/tbody/tr/td[@nowrap='nowrap']/a"):
#    display(p.get_attribute("href"))

In [163]:
#url = "https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20181216/ST/4"


"""
url_list = []
current_i = 0
for i in range(10): #(730):
    current_i = i
    target_date = datetime.datetime.now() - datetime.timedelta(days=i)
    driver.get(url.format(target_date.strftime("%Y%m%d"))) 
    time.sleep(5)
    element = WebDriverWait(driver, 10).until(lambda x: driver.find_elements_by_xpath("//*[@id='results']/div[2]/table/tbody/tr/td[@nowrap='nowrap']/a"))
    
    for p in element:
        url_list.append(p.get_attribute("href"))
    
print(len(url_list))
"""

'\nurl_list = []\ncurrent_i = 0\nfor i in range(10): #(730):\n    current_i = i\n    target_date = datetime.datetime.now() - datetime.timedelta(days=i)\n    driver.get(url.format(target_date.strftime("%Y%m%d"))) \n    time.sleep(5)\n    element = WebDriverWait(driver, 10).until(lambda x: driver.find_elements_by_xpath("//*[@id=\'results\']/div[2]/table/tbody/tr/td[@nowrap=\'nowrap\']/a"))\n    \n    for p in element:\n        url_list.append(p.get_attribute("href"))\n    \nprint(len(url_list))\n'

In [164]:
#url_list

['https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/2',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/3',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/4',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/5',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/6',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/7',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/8',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/9',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/10',
 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20190101/ST/11']

In [427]:
# Date and track
# //*[@id="results"]/div[3]/table/tbody/tr/td[1]

# //*[@id="results"]/div[5]/div[2]/table

def check_exists_by_xpath(page, xpath):
    try:
        page.find_element_by_xpath(xpath)
    except NoSuchElementException:
        if debug:
            logging.info('Debug: ', xpath, page.current_url)
        #print("Debug: ", xpath, page.current_url)
        return False
    return True


def get_race_profile(page):
    res = pd.read_html(page.find_element_by_xpath("//*[@id='results']/div[5]/div[2]/table").get_attribute("outerHTML"))[0]
    #display(res)
    column_name = ["Race_type", "Going", "Handicap", "Course", \
                   "Winnings", "Run_Time", "Sectional_time"]
    #display(res)
    result = []
    result.append(res.iloc[0,0])
    result.append(res.iloc[0,2])
    result.append(res.iloc[1,0])
    result.append(res.iloc[1,2])
    result.append(res.iloc[2,0])
    result.append(res.iloc[2,2:])
    #result.append(res.iloc[2,3])
    #result.append(res.iloc[2,4])
    #result.append(res.iloc[2,5])
    result.append(res.iloc[3,2:])
    #result.append(res.iloc[3,3])
    #result.append(res.iloc[3,4])
    #result.append(res.iloc[3,5])

    #display(result)
    result_pd = pd.DataFrame([result], columns=column_name)
    return result_pd
#display(get_race_profile(driver))

def get_race_date_track(page):
    if check_exists_by_xpath(page, '//*[@id="results"]/div[3]/table/tbody/tr/td[1]'):
        race_date = page.find_element_by_xpath('//*[@id="results"]/div[3]/table/tbody/tr/td[1]').text
        #isplay(race_date)
        re_date_match = re.search(r"Race Meeting: (\d{2}/\d{2}/\d{4})\s+(.*)$", race_date)
        #isplay(re_date_match.group(2))
        return (re_date_match.group(1), re_date_match.group(2).strip())
    return None

def get_race_number(page):
    if check_exists_by_xpath(page, '//*[@id="results"]/div[5]/div[1]'):
        race_number = page.find_element_by_xpath('//*[@id="results"]/div[5]/div[1]').text
        re_match = re.search(r"RACE (\d+) \((\d+)\)$", race_number)
        if re_match:
            return re_match.group(2)
    return None

        
#get_race_date_track(driver)
#get_race_number(driver)



In [428]:
#res = pd.read_html(driver.find_element_by_xpath("//*[@id='results']/div[6]/table").get_attribute("outerHTML"))[0]

#display(res)


def get_horse_placement(page):
    result = {}
    for i in range(15):
        result[i] = []
    
    for k in page.find_elements_by_xpath("//*[@id='results']/div[6]/table/tbody/tr"):
        for i, j in enumerate(k.find_elements_by_xpath("td")):
            if i == 0:
                result[i].append(j.text)
            if i == 1:
                result[i].append(j.text)
            if i == 2:
                result[i].append(j.text)
                if check_exists_by_xpath(j, "a"):
                    result[12].append(j.find_element_by_xpath("a").get_attribute("href"))
                else:
                    result[12].append("")
            if i == 3:
                result[i].append(j.text)
                if check_exists_by_xpath(j, "a"):
                    result[13].append(j.find_element_by_xpath("a").get_attribute("href"))
                else:
                    result[13].append("")
            if i == 4:
                result[i].append(j.text)
                if check_exists_by_xpath(j, "a"):
                    result[14].append(j.find_element_by_xpath("a").get_attribute("href"))
                else:
                    result[14].append("")
            if i == 5:
                result[i].append(j.text)
            if i == 6:
                result[i].append(j.text)
            if i == 7:
                result[i].append(j.text)
            if i == 8:
                result[i].append(j.text)
            if i == 9:
                result[i].append(j.text)
            if i == 10:
                result[i].append(j.text)
            if i == 11:
                result[i].append(j.text)
            #display(j.get_attribute("innerHTML")
    #display(result)
    res = pd.DataFrame(result) #, columns=["pla", "horse_no", "horse", "jockey", "trainer", "actual_wt", "declare_horse_wt", "draw", "LBW", "running_position", "finish_time", "win_odds", "horse_url", "jockey_url", "trainer_url"]))
    res.columns = ["pla", "horse_no", "horse", "jockey", "trainer", "actual_wt", "declare_horse_wt", "draw", "LBW", "running_position", "finish_time", "win_odds", "horse_url", "jockey_url", "trainer_url"]
    #display(res)
    return res
#get_horse_placement(driver)

def get_dividend(page):
    xpath_list = ["tbody/tr[3]/td[2]", "tbody/tr[3]/td[3]", "tbody/tr[4]/td[2]", "tbody/tr[4]/td[3]", "tbody/tr[5]/td[1]", "tbody/tr[5]/td[2]", \
                 "tbody/tr[6]/td[1]", "tbody/tr[6]/td[2]", "tbody/tr[7]/td[2]", "tbody/tr[7]/td[3]", "tbody/tr[8]/td[2]", "tbody/tr[8]/td[3]", \
                 "tbody/tr[9]/td[1]", "tbody/tr[9]/td[2]", "tbody/tr[10]/td[1]", "tbody/tr[10]/td[2]", "tbody/tr[11]/td[2]", "tbody/tr[11]/td[3]", \
                 "tbody/tr[12]/td[1]", "tbody/tr[12]/td[2]", "tbody/tr[13]/td[1]", "tbody/tr[13]/td[2]", "tbody/tr[15]/td[2]", "tbody/tr[15]/td[3]", \
                 "tbody/tr[16]/td[2]", "tbody/tr[16]/td[3]", "tbody/tr[17]/td[2]", "tbody/tr[17]/td[3]", "tbody/tr[18]/td[2]", "tbody/tr[18]/td[3]"]
    column_names = ["win_combination", "win_dividend", "place_combination1", "place_dividend1", "place_combination2", "place_dividend2", \
                   "place_combination3", "place_dividend3", "quinella_combination", "quinella_dividend", "quinella_place_combination1", "quinella_place_dividend1", \
                   "quinella_place_combination2", "quinella_place_dividend2", "quinella_place_combination3", "quinella_place_dividend3", \
                   "three_pick_one_combination1", "three_pick_one_dividend1", "three_pick_one_combination2", "three_pick_one_dividend2",\
                   "three_pick_one_combination3", "three_pick_one_dividend3", "tierce_combination", "tierce_dividend", \
                   "trio_combination", "trio_dividend", "first_four_combination", "first_four_dividend", "quartet_combination",\
                   "quartet_dividend"]
    results = []
    if check_exists_by_xpath(page, '//*[@id="results"]/div[7]/table/tbody/tr/td[1]/table'):
        p = page.find_element_by_xpath('//*[@id="results"]/div[7]/table/tbody/tr/td[1]/table')
        #display(p.text[:8])
        if "Dividend" == p.text[:8]:
            results = []
            #display()
            for x in xpath_list:
                #display(x)
                if check_exists_by_xpath(p, x):
                    results.append(p.find_element_by_xpath(x).text)
                else:
                    results.append("")
    #display(results)
    return pd.DataFrame([results], columns= column_names)
            

    
#get_dividend(driver)

In [431]:

def get_race_info(page, race_url, race_number):
    if check_exists_by_xpath(page, '//*[@id="results"]/div[5]/div[1]'):
        #race_number = get_race_number(page)
        date, track = get_race_date_track(page)
        df1 = pd.DataFrame([[race_number, date, track, race_url]], columns=["race_number", "date", "track", "race_url"] )
        df2 = get_race_profile(page)
        df3 = get_dividend(page)
        df4 = pd.concat([df1,df2, df3], axis=1)
        df4["update_datetime"] = datetime.datetime.now()
        #df4["race_id"] = race_url
        df5 = get_horse_placement(page)
        df5["race_url"] = race_url
        df5["race_number"] = race_number
        df5["race_date"] = date
        df5["race_track"] = track
        df5['update_datetime'] = datetime.datetime.now()
        #display(res4["race_url"])
        return (df4, df5)
    else:
        return None
#get_race_info(driver, "rac", "asdf")

#get_horse_placement(driver)
#pd.DataFrame(res).T
#display(res)

In [430]:
def save_url_progress(location, url_list):
    if os.path.isfile(location):
        with open(location, "r+b") as f:
            try:
                #urls = pickle.load(f)
                #display(len(urls))
                #urls.update(url_list)
                #display(len(urls))
                #display(urls)
                #urls = {**urls, **url_list}
                #display(urls)
                pickle.dump(url_list)
                return urls
            except Exception:
                pass
    with open(location, "wb") as f:
        pickle.dump(url_list, f)
    return url_list
    
    #display(concat_list)
#save_url_progress("data/race_urls.pickle", ["asdfg", "asdg" , "asdgsdg", "asdg", "sdfdsg", "gasdasdg"])
    #.to_pickle("data/race_urls.pickle")
    
def open_url_progress(location):
    if os.path.isfile(location):
        with open(location, "rb") as f:
            try:
                urls = pickle.load(f)
                return urls
            except Exception:
                pass
    else:
        return None

#open_url_progress("data/race_urls.pickle")



In [423]:
#url = "https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20181216/ST/4"
url = "https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/{}"
driver.implicitly_wait(4)


url_list = open_url_progress("data/race_urls.pickle")
#url_list = {}
if not url_list:
    url_list = {}
    
    
def update_dict(url_dict, new_url, race_id):
    if new_url in url_dict:
        url_dict[new_url] = {"create_date":datetime.date.today(), "race_number": race_id, "update_date": None}
    else:
        url_dict[new_url] = {"create_date":datetime.date.today(), "race_number": race_id, "update_date": None}
    return url_dict
    
current_i = 0
for i in range(1000,5000):
    current_i = i
    target_date = datetime.datetime.now() - datetime.timedelta(days=i)
    #display(target_date.strftime("%Y%m%d"))
    target_url = url.format(target_date.strftime("%Y%m%d"))
    #display(target_url)
    driver.get(target_url) 
    driver.implicitly_wait(4)
    try:
        element = WebDriverWait(driver, 10).until(lambda x: EC.presence_of_element_located((By.ID, "mainContainer"))) #driver.find_elements_by_xpath("//*[@id='results']/div[2]/table/tbody/tr/td[@nowrap='nowrap']/a"))
    except e:
        if debug:
            logging.info("Page didn't load: ", driver.current_url)
        #print("error")
    finally:
        pass
    
    if check_exists_by_xpath(driver, '//*[@id="results"]/div[5]/div[1]'): 
        #print("Page found ")
        race_number = get_race_number(driver)
        url_list = update_dict(url_list, url, race_number) 
        element = WebDriverWait(driver, 10).until(lambda x: driver.find_elements_by_xpath("//*[@id='results']/div[2]/table/tbody/tr/td[@nowrap='nowrap']/a"))
        #print(element)
        for p in element: #driver.find_elements_by_xpath("//*[@id='results']/div[2]/table/tbody/tr/td[@nowrap='nowrap']/a"):
            new_url = p.get_attribute("href")
            #print(url)
            url_list = update_dict(url_list, new_url, race_number)
    if (i+1) % 50 ==  0:
        display(target_date)
        save_url_progress("data/race_urls.pickle", url_list)
        #url_list = {}
    #print(url_list)
save_url_progress("data/race_urls.pickle", url_list)    
print(len(url_list))
#print(current_i)
print(target_date)

datetime.datetime(2016, 2, 21, 1, 6, 21, 193808)

datetime.datetime(2016, 1, 2, 1, 10, 23, 372333)

datetime.datetime(2015, 11, 13, 1, 14, 21, 634456)

datetime.datetime(2015, 9, 24, 1, 18, 20, 707602)

datetime.datetime(2015, 8, 5, 1, 22, 16, 992098)

...

datetime.datetime(2006, 1, 4, 6, 0, 30, 992555)

datetime.datetime(2005, 11, 15, 6, 4, 27, 246841)

datetime.datetime(2005, 9, 26, 6, 8, 25, 785268)

datetime.datetime(2005, 8, 7, 6, 12, 26, 398006)

KeyboardInterrupt: 

In [212]:
#save_url_progress("data/race_urls.pickle", url_list)

In [432]:
def update_race(location, new_races):
    if os.path.isfile(location):
        races = pd.read_pickle(location)
        #races = races.merge(new_races, how="left", on=["date", "race_number"])
        races = pd.concat([races, new_races], axis=0, ignore_index=True)
        races.to_pickle(location)
    else:
        new_races.to_pickle(location)

def update_race_placement(location, new_race_placements):
    if os.path.isfile(location):
        placements = pd.read_pickle(location)
        #placements = placements.merge(new_race_placements, how="left", on=["date", "race_number", "horse_no"])
        placements = pd.concat([placements, new_race_placements], axis=0, ignore_index=True)
        placements.to_pickle(location)
    else:
        new_race_placements.to_pickle(location)
        
        
def merge_table_ifexist(table1, table2, on):
    if len(table1) != 0:
        return pd.concat([table1, table2], axis=0)
    else:
        return table2
    
    
    

In [410]:

#race_url_dict = {}
#url_list2 = {}
#for race_url, value in url_list.items():
#    url_list2[race_url] = {"created_date": value[0], "race_number": value[1], "updated_date": None}
    
#save_url_progress("data/race_urls.pickle", url_list2)
#url_list3 = open_url_progress("data/race_urls.pickle")
#print(url_list)

"""
url_list = open_url_progress("data/race_urls.pickle")
for u, v in url_list.items():
    if "updated_date" in v:
        url_list[u]["update_date"] = v["updated_date"]
        print(v['create_date'], v["updated_date"], url_list[u]["update_date"])
    else:
        print(v["update_date"])
    
save_url_progress("data/race_urls.pickle", url_list)    
"""


#url_list = open_url_progress("data/race_urls.pickle")
#url_list.pop('https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/{}', None)
#save_url_progress("data/race_urls.pickle", url_list) 
#'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/{}' in url_list


False

In [434]:

#url_list
url_list = open_url_progress("data/race_urls.pickle")
#display(url_list2)
#display(url_list)
driver.implicitly_wait(4)
races_table = pd.DataFrame()
placement_table = pd.DataFrame()
counter = 0
for race_url, value in url_list.items():
    #display(value["update_date"])
    if value["update_date"] == None or value["update_date"] < datetime.datetime.now() - datetime.timedelta(days=30):
        if 'https://racing.hkjc.com/racing/Info/Meeting/Results/English/Simulcast/' == race_url[:70]:
            url_list[race_url]["update_date"] = datetime.datetime.now() + datetime.timedelta(days=365)
        else:
            #display(race_url)
            driver.get(race_url)
            race_number = get_race_number(driver)
            if race_number:
                race, placement = get_race_info(driver, race_url, race_number)
                #display(races_table, race)
                races_table = merge_table_ifexist(races_table, race, ["date", "race_number"]) #races_table.merge(race, how="left", on=["date", "race_number"])
                #display(placement_table)
                placement_table = merge_table_ifexist(placement_table, placement, ["race_date", "race_number", "horse_no"])#placement_table.merge(placement, how="left", on=['date', 'race_number', 'horse_no'])
                url_list[race_url]["update_date"] = datetime.datetime.now() 
                counter += 1
                if (counter) % 50 == 0:
                    display(races_table.shape, placement_table.shape)
                    update_race("data/race_table.pickle", races_table)
                    update_race_placement("data/race_placements.pickle", placement_table)
                    races_table = pd.DataFrame()
                    placement_table = pd.DataFrame()
                    save_url_progress("data/race_urls.pickle", url_list)
            else:
                print("Race Number not found", race_url)
                if driver.current_url.split("?")[0] == "https://racing.hkjc.com/racing/overseas/English/Results.aspx":
                    url_list[race_url]["update_date"] = datetime.datetime.now() + datetime.timedelta(days=365)
                
                
update_race("data/race_table.pickle", races_table)
update_race_placement("data/race_placements.pickle", placement_table)
save_url_progress("data/race_urls.pickle", url_list);

Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20161026/HV/4
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/{}
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20160328/ST/9
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20160124/ST/4
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20160113/HV/2
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20160113/HV/4
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20160101/ST/6
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20151213/ST/6
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20151114/ST/10
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/2015

(50, 42)

(645, 20)

Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150701/ST/11
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150520/HV/8
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150516/ST/5
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150516/ST/7
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150513/HV/4
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150509/ST/6
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150422/HV/4
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150419/ST/9
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20150412/ST/7
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English

(50, 42)

...

Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20091202/HV/2
Race Number not found https://www.hkjc.com/english/racing/results.asp?raceDate=29/11/2009&raceno=1&venue=S1
Race Number not found https://racing.hkjc.com/racing/Info/Meeting/Results/English/Local/20091115/ST/2


(50, 42)

(656, 20)

Race Number not found https://www.hkjc.com/english/racing/results.asp?raceDate=07/11/2009&raceno=1&venue=S1


TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.80)
  (Driver info: chromedriver=71.0.3578.80,platform=Linux 4.15.0-43-generic x86_64)


In [347]:

#races_table.merge(race, how="left", on=["date", "race_number"])
#race

get_horse_placement(driver)


ValueError: arrays must all be same length